# Exp M3b: Low λ Exploration for η=0.8

## 目的
M3（λ=0.38-0.50）ではη=0.8がすでにCritical状態だった。
より低いλでOrdered領域が存在するか、最適λを探索する。

## 実験設計
- **λ**: [0.15, 0.20, 0.25, 0.30, 0.35]
- **η（ノイズ率）**: 0.8（固定）
- **Seeds**: 0-4（5個）
- **Total**: 5 × 5 = **25 runs**

## 推定時間
~25 runs × 10 min ≈ **4 時間**

In [ ]:
# ===== セットアップ =====
from google.colab import drive
drive.mount('/content/drive')

import os
import glob
from datetime import datetime

EXP_NAME = 'exp_M3b_low_lambda_eta08'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

# 既存フォルダがあれば再利用（チェックポイント対応）
existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    TIMESTAMP = SAVE_DIR.split('_')[-2] + '_' + SAVE_DIR.split('_')[-1]
    print(f'🔄 Resuming from: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New experiment: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)

print(f'Experiment: {EXP_NAME}')
print(f'Timestamp: {TIMESTAMP}')
print(f'Save directory: {SAVE_DIR}')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
import numpy as np
import json
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
# ===== モデル定義 =====
class IndexedDataset(Dataset):
    """インデックスも返すDataset wrapper"""
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes)
            )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)

class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, 2, 1)
        self.layer2 = self._make_layer(128, 2, 2)
        self.layer3 = self._make_layer(256, 2, 2)
        self.layer4 = self._make_layer(512, 2, 2)
        self.linear = nn.Linear(512, num_classes)
    
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for s in strides:
            layers.append(BasicBlock(self.in_planes, planes, s))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        return self.linear(out)

In [ ]:
# ===== 実験パラメータ =====
BATCH_SIZE = 256
NUM_WORKERS = 4
EPOCHS = 100
LR = 0.1
K = 16  # Value gradient update frequency

# M3b: 低λ探索（η=0.8）
LAMBDAS = [0.15, 0.20, 0.25, 0.30, 0.35]
NOISE_RATE = 0.8
SEEDS = [0, 1, 2, 3, 4]

# 実験条件リスト生成
experiments = []
for lam in LAMBDAS:
    for seed in SEEDS:
        experiments.append({'lambda': lam, 'seed': seed})

print(f'Lambdas: {LAMBDAS}')
print(f'Noise rate: {NOISE_RATE}')
print(f'Seeds: {SEEDS}')
print(f'Total experiments: {len(experiments)}')
print(f'Estimated time: {len(experiments) * 10 / 60:.1f} hours')

In [ ]:
# ===== ユーティリティ関数 =====
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

def load_cifar10():
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    trainset = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform_test)
    return trainset, testset

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    labels = np.array(labels)
    n_samples = len(labels)
    n_noisy = int(noise_rate * n_samples)
    noisy_idx = np.random.choice(n_samples, n_noisy, replace=False)
    noisy_labels = labels.copy()
    for idx in noisy_idx:
        choices = [l for l in range(10) if l != labels[idx]]
        noisy_labels[idx] = np.random.choice(choices)
    return noisy_labels

def get_data_loaders(trainset, testset):
    train_loader = DataLoader(IndexedDataset(trainset), batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE,
                             shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
    return train_loader, test_loader

def evaluate(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

In [ ]:
# ===== Dual-Gradient Learning =====
def train_dual_gradient(model, train_loader, test_loader, clean_labels, noisy_labels, lam):
    """
    Dual-Gradient Learning with correct index-based label access.
    
    g_struct = noisy labels の勾配（データに従う力）
    g_value = clean labels の勾配（価値方向）
    g_mix = (1-λ) × ĝ_struct + λ × ĝ_value
    """
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 75], gamma=0.1)
    criterion = nn.CrossEntropyLoss()
    
    clean_labels_tensor = torch.tensor(clean_labels, device=device)
    noisy_labels_tensor = torch.tensor(noisy_labels, device=device)
    
    cached_g_value = None
    global_step = 0
    
    best_acc = 0
    cos_history = []
    history = {'epoch': [], 'test_acc': [], 'test_error': []}
    
    for epoch in range(EPOCHS):
        model.train()
        epoch_cos = []
        
        for inputs, _, indices in train_loader:  # ← IndexedDatasetからindicesを取得
            inputs = inputs.to(device, non_blocking=True)
            indices = indices.to(device, non_blocking=True)
            batch_noisy = noisy_labels_tensor[indices]  # ← 正しいインデックスでアクセス
            batch_clean = clean_labels_tensor[indices]  # ← 正しいインデックスでアクセス
            
            # Structure gradient (from noisy labels)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss_struct = criterion(outputs, batch_noisy)
            loss_struct.backward(retain_graph=True)
            g_struct = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Value gradient (from clean labels)
            if global_step % K == 0 or cached_g_value is None:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss_value = criterion(outputs, batch_clean)
                loss_value.backward()
                cached_g_value = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Normalize and mix
            g_struct_norm = g_struct / (g_struct.norm() + 1e-12)
            g_value_norm = cached_g_value / (cached_g_value.norm() + 1e-12)
            
            cos_sim = (g_struct_norm @ g_value_norm).item()
            epoch_cos.append(cos_sim)
            
            g_mix = (1 - lam) * g_struct_norm + lam * g_value_norm
            
            optimizer.zero_grad()
            idx = 0
            for p in model.parameters():
                numel = p.numel()
                p.grad = g_mix[idx:idx+numel].view(p.shape).clone()
                idx += numel
            optimizer.step()
            global_step += 1
        
        scheduler.step()
        cos_history.append(np.mean(epoch_cos))
        
        if (epoch + 1) % 10 == 0:
            acc = evaluate(model, test_loader)
            best_acc = max(best_acc, acc)
            history['epoch'].append(epoch + 1)
            history['test_acc'].append(acc)
            history['test_error'].append(1 - acc)
    
    final_acc = evaluate(model, test_loader)
    avg_cos = np.mean(cos_history)
    
    return final_acc, max(best_acc, final_acc), avg_cos, history

In [ ]:
# ===== データ準備 =====
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader, test_loader = get_data_loaders(trainset, testset)

print('Data prepared')

# GPU warmup
warmup_model = ResNet18().to(device)
for _ in range(20):
    _ = warmup_model(torch.randn(BATCH_SIZE, 3, 32, 32, device=device))
del warmup_model
torch.cuda.empty_cache()
print('GPU warmed up')

In [ ]:
# ===== メイン実験ループ =====
results = []
checkpoint_file = f'{SAVE_DIR}/{EXP_NAME}_checkpoint.json'
completed = set()

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        results = json.load(f)
    for r in results:
        completed.add((r['lambda'], r['seed']))
    print(f'Checkpoint loaded: {len(results)} runs completed')

total = len(experiments)
start_idx = len(completed)

for i, exp in enumerate(experiments):
    lam, seed = exp['lambda'], exp['seed']
    
    if (lam, seed) in completed:
        continue
    
    run_num = len(completed) + 1
    print(f'\n[{run_num}/{total}] λ={lam} η={NOISE_RATE} seed={seed}')
    
    set_seed(seed)
    noisy_labels = inject_label_noise(clean_labels, NOISE_RATE, seed)
    
    model = ResNet18().to(device)
    
    start_time = time.time()
    final_acc, best_acc, avg_cos, history = train_dual_gradient(
        model, train_loader, test_loader, clean_labels, noisy_labels, lam
    )
    elapsed = time.time() - start_time
    
    result = {
        'experiment_id': f'M3b-{run_num:03d}',
        'experiment': EXP_NAME,
        'lambda': lam,
        'noise_rate': NOISE_RATE,
        'seed': seed,
        'test_acc': final_acc,
        'test_error': 1 - final_acc,
        'best_acc': best_acc,
        'best_error': 1 - best_acc,
        'avg_cos_struct_value': avg_cos,
        'time_seconds': elapsed,
        'history': history
    }
    results.append(result)
    completed.add((lam, seed))
    
    status = '✅ EXCELLENT' if best_acc > 0.85 else ('⚠️ COLLAPSE' if best_acc < 0.5 else '')
    print(f'  Error: {1-best_acc:.4f} | cos: {avg_cos:.4f} | Time: {elapsed/60:.1f} min {status}')
    
    # Save checkpoint
    with open(checkpoint_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    remaining = total - run_num
    eta = remaining * elapsed / 3600
    print(f'  Progress: {run_num}/{total} | ETA: {eta:.1f} hours')
    
    del model
    torch.cuda.empty_cache()

print('\n' + '='*70)
print('ALL EXPERIMENTS COMPLETED!')
print('='*70)

In [ ]:
# ===== 結果保存 =====
import pandas as pd

with open(f'{SAVE_DIR}/{EXP_NAME}_results.json', 'w') as f:
    json.dump(results, f, indent=2)

results_flat = [{k: v for k, v in r.items() if k != 'history'} for r in results]
df = pd.DataFrame(results_flat)
df.to_csv(f'{SAVE_DIR}/{EXP_NAME}_results.csv', index=False)

print(f'Results saved to {SAVE_DIR}')
print('\nSummary by λ:')
print(df.groupby('lambda')[['best_error', 'avg_cos_struct_value']].agg(['mean', 'std']))

In [ ]:
# ===== 可視化 =====
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左: Error vs λ
ax = axes[0]
stats = df.groupby('lambda')['best_error'].agg(['mean', 'std'])
ax.errorbar(stats.index, stats['mean'], yerr=stats['std'],
            marker='o', capsize=4, linewidth=2, markersize=8, color='red')
ax.axhline(y=0.20, color='gray', linestyle='--', alpha=0.5, label='M3 lowest (λ=0.38)')
ax.set_xlabel('λ (Value gradient weight)', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title(f'M3b: Low λ Exploration (η={NOISE_RATE})', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

# 右: Cosine vs λ
ax = axes[1]
stats = df.groupby('lambda')['avg_cos_struct_value'].agg(['mean', 'std'])
ax.errorbar(stats.index, stats['mean'], yerr=stats['std'],
            marker='s', capsize=4, linewidth=2, markersize=8, color='blue')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('λ (Value gradient weight)', fontsize=12)
ax.set_ylabel('cos(g_struct, g_value)', fontsize=12)
ax.set_title('Gradient Alignment', fontsize=14)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{EXP_NAME}_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print('Figure saved!')

In [ ]:
# ===== M1/M3との統合比較 =====
print('='*70)
print('M3b RESULTS: Low λ for η=0.8')
print('='*70)
print(f'{"λ":>6} | {"error":>8} | {"std":>6} | {"cos":>8} | state')
print('-' * 55)

for lam in LAMBDAS:
    subset = df[df['lambda'] == lam]
    mean_err = subset['best_error'].mean()
    std_err = subset['best_error'].std()
    mean_cos = subset['avg_cos_struct_value'].mean()
    
    if mean_err < 0.20:
        state = '✅ Ordered'
    elif mean_err < 0.30:
        state = '⚡ Critical'
    else:
        state = '❌ Collapse'
    
    print(f'{lam:>6.2f} | {mean_err:>7.1%} | {std_err:>5.1%} | {mean_cos:>+8.3f} | {state}')

print('\n--- Comparison with M1 (η=0.8) ---')
print('M1: λ=0.30-0.40 showed best error ~23-24%')
print('M3b explores if lower λ can achieve Ordered state')